<a href="https://colab.research.google.com/github/Wanita-8943/efficientnet_keras_transfer_learning/blob/master/All.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import shutil

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv (r'/content/drive/My Drive/All_Film.csv')
df

,Class,Filename,Path_filename,Sex,Floder
0,Y7,V1.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
1,Y7,V2.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
2,Y7,V3.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
3,Y7,V6.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
4,Y7,V7.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
...,...,...,...,...,...
1836,Y18,S264.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both
1837,Y18,S268.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both
1838,Y18,S270.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both
1839,Y18,S274.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both


In [5]:
classes = ['Y7','Y8','Y9','Y10','Y11','Y12','Y13','Y16','Y17','Y18']
len(classes)

10

In [6]:
df.shape

(1841, 5)

In [12]:
print(set(df['Class']))
print(set(df['Sex']))
print(set(df['Floder']))

{'Y7', 'Y13', 'Y10', 'Y17', 'Y9', 'Y11', 'Y12', 'Y16', 'Y8', 'Y18'}
{'เพศชาย', 'เพศหญิง'}
{'Rt', 'Both', 'Lt'}


In [16]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import os
import glob
import shutil
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

In [23]:
#กำหนดค่าพารามิเตอร์
batch_size = 15

width = 150
height = 150
epochs = 20
NUM_TRAIN = 1800
NUM_TEST = 200
dropout_rate = 0.2
input_shape = (height, width, 3)

#Clone efficientnet repo


In [13]:
#ดึงข้อมูลใน Github มาใช้
import os
%cd /content
if not os.path.isdir("efficientnet_keras_transfer_learning"):
 !git clone https://github.com/Wanita-8943/efficientnet_keras_transfer_learning
%cd efficientnet_keras_transfer_learning/


/content
Cloning into 'efficientnet_keras_transfer_learning'...
remote: Enumerating objects: 254, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 254 (delta 42), reused 10 (delta 5), pack-reused 178
Receiving objects: 100% (254/254), 6.77 MiB | 13.68 MiB/s, done.
Resolving deltas: 100% (147/147), done.
/content/efficientnet_keras_transfer_learning


In [14]:
# Options: EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3
# Higher the number, the more complex the model is.
from efficientnet import EfficientNetB0 as Net
from efficientnet import center_crop_and_resize, preprocess_input

In [18]:
# loading pretrained conv base model
# โหลดโมเดล มาโดยตัด output ของโมเดลออก เเต่ยังใช้ input อันเดิม
# เเละโหลด weight ของโมเดล มาด้วยที่ชื่อว่า imagenet
conv_base = Net(weights='imagenet', include_top=False, input_shape=input_shape)

16717576/16717576 [==============================] - 1s 0us/step


In [21]:
base_dir = '/content/drive/My Drive/TVT'
os.makedirs(base_dir, exist_ok=True)

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir, exist_ok=True)
validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir, exist_ok=True)
test_dir = os.path.join(base_dir, 'test')
os.makedirs(test_dir, exist_ok=True)